In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

Reading the wikipedia html table into a dataframe

In [40]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', match='Toronto')[0]

In [42]:
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Removing cells that have no assigned borough

In [61]:
df = df[df['Borough'] != 'Not assigned'].copy()

Checking if any neighbourhoods are not assigned

In [62]:
df[df['Neighbourhood'] == 'Not assigned']

,Postal Code,Borough,Neighbourhood


Dropping any duplicate Postal Codes

In [63]:
df.drop_duplicates(subset=['Postal Code'], inplace=True)

Getting the shape of the dataframe which shows the row count

In [64]:
df.shape

(103, 3)